In [10]:
# %load plot_template.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

plt.rc('font', family='serif')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10, color='grey')
plt.rc('ytick', labelsize=10, color='grey')
plt.rc('legend', fontsize=12, loc='lower left')
plt.rc('figure', titlesize=12)
plt.rc('savefig', dpi=330, bbox='tight')
%matplotlib inline

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [11]:
## Replace PATH_TO_ROOT

import sys
PATH_TO_ROOT = '/home/aistudio/results'
sys.path.insert(0,PATH_TO_ROOT)

In [12]:
import utlis
from sklearn.metrics import mean_absolute_error, mean_squared_error

import lightgbm as lgb

In [13]:
def generate_fig_path(filename):
    fig_dir = '/home/aistudio/results'
    os.makedirs(fig_dir, exist_ok=True)
    return os.path.join(fig_dir, filename)

def generate_input_data_path(data_name):
    '''
    Generate the path to input data
    '''
    return '{}/results/data/{}.csv'.format(PATH_TO_ROOT, data_name)

def generate_result_data_path(data_name):
    '''
      Generate the path to input data
    '''
    return '{}/{}.csv'.format(PATH_TO_ROOT, data_name)

In [14]:
!pip install lightgbm


Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/


In [15]:
!pip install holidays
!pip install chinese-calendar


Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.9/917.9 kB 43.7 MB/s eta 0:00:00
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/


In [22]:
def add_holiday_auto(data, city_name):
    """
    Add holiday column based on city-specific calendar:
    - China: uses 'chinese_calendar' (accurate official holiday + working weekend)
    - Singapore & US: holidays + weekends
    - Others: weekends only

    Parameters:
        data (pd.DataFrame): input with datetime index
        city_name (str): e.g. 'fuzhou', 'singapore', 'ny'

    Returns:
        pd.DataFrame: with binary 'Holiday' column
    """
    import holidays
    data = data.copy()
    city = city_name.lower()

    if city in ['fuzhou', 'beijing', 'shanghai', 'china']:
        try:
            import chinese_calendar
            # Use accurate national holiday schedule
            data['Holiday'] = data.index.map(chinese_calendar.is_holiday).astype(int)
        except ImportError:
            print("[Warning] 'chinese_calendar' not installed. Fallback to basic China holidays + weekends.")
            cn_holidays = holidays.China(years=range(data.index.year.min(), data.index.year.max()+1))
            weekend_flag = (data.index.weekday >= 5).astype(int)
            holiday_flag = pd.to_datetime(data.index.date).isin(cn_holidays).astype(int)
            data['Holiday'] = (holiday_flag | weekend_flag)

    elif city in ['singapore', 'sg']:
        sg_holidays = holidays.Singapore(years=range(data.index.year.min(), data.index.year.max()+1))
        holiday_flag = pd.to_datetime(data.index.date).isin(sg_holidays).astype(int)
        weekend_flag = (data.index.weekday >= 5).astype(int)
        data['Holiday'] = (holiday_flag | weekend_flag)

    elif city in ['ny', 'la', 'new york', 'los angeles', 'sacramento', 'sac', 'us']:
        from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
        cal = calendar()
        us_holidays = cal.holidays(start=data.index.min(), end=data.index.max())
        holiday_flag = data.index.isin(us_holidays).astype(int)
        weekend_flag = (data.index.weekday >= 5).astype(int)
        data['Holiday'] = (holiday_flag | weekend_flag)

    else:
        print(f"[Warning] Unknown city '{city_name}'. Fallback to weekends only.")
        data['Holiday'] = (data.index.weekday >= 5).astype(int)

    return data


In [17]:
import pandas as pd
import lightgbm as lgb
import os
from sklearn.metrics import mean_squared_error


def lightGBM_train(region, params):
    # 定义修改后的文件路径
    file_path = f'/home/aistudio/data/{region}_hourly_modified.xlsx'
    
    # 读取修改后的数据（假设Datetime是索引）
    data = pd.read_excel(file_path, index_col=0, parse_dates=True)
    data.index = pd.to_datetime(data.index, dayfirst=True)
    data.sort_index(inplace=True)  # 确保索引排序

    # 根据数据实际情况进行截取（例如，截取到2019-07-01）
    data = data.truncate(after='2019-07-01')
    data = add_holiday_auto(data, region)
   
    # 添加时间和节假日特征
    data['Month'] = data.index.month
    data['dayOfWeek'] = data.index.weekday



    # 添加滞后负荷和滑动平均
    data['load_lag1'] = data['load'].shift(1)
    data['load_avg3'] = data['load'].rolling(window=3).mean()

    # 删除缺失值
    data_lgmb = data[['Temperature', 'Humidity', 'load', 'Holiday', 'dayOfWeek', 'Month', 'load_lag1', 'load_avg3']].dropna()

    # 特征与目标
    feature_cols = ['Temperature', 'Humidity', 'Holiday', 'dayOfWeek', 'Month', 'load_lag1', 'load_avg3']
    target_col = 'load'

    # 划分训练测试集
    train_size = int(len(data_lgmb) * 0.75)
    data_train = data_lgmb.iloc[:train_size]
    data_test = data_lgmb.iloc[train_size:]

    X_train = data_train[feature_cols].values
    y_train = data_train[target_col].values
    X_test = data_test[feature_cols].values
    y_test = data_test[target_col].values
    X_all = data_lgmb[feature_cols].values

    # 训练LightGBM模型，添加验证集与early stopping
    d_train = lgb.Dataset(X_train, categorical_feature=[2, 3, 4], label=y_train)
    d_valid = lgb.Dataset(X_test, label=y_test)
    regr = lgb.train(params, d_train, num_boost_round=5000)




        # 计算并打印误差
    rmse_train = mean_squared_error(regr.predict(X_train), y_train)**0.5
    rmse_test = mean_squared_error(regr.predict(X_test), y_test)**0.5
    cvrmse_train = rmse_train / y_train.mean()
    cvrmse_test = rmse_test / y_test.mean()

    print(f'-------City: {region}--------------')
    print(f'RMSE for Train: {rmse_train}')
    print(f'RMSE for Test: {rmse_test}')
    print(f'CVRMSE for Train: {cvrmse_train:.4f}')
    print(f'CVRMSE for Test: {cvrmse_test:.4f}')
    # 保存预测结果（包含原始目标列、预测值）
    data_lgmb['yhat_lgbm'] = regr.predict(X_all)
    data_lgmb.rename(columns={'load': 'Electricity demand, daily sum, (GWh)'}, inplace=True)
    # 保存到 /home/aistudio/results
    result_path = generate_result_data_path(f'lgbm_{region}.csv')
    data_lgmb.to_csv(result_path)
    # 获取特征名称（你传给模型的列名）
    feature_names = ['Temperature', 'Humidity', 'Holiday', 'dayOfWeek', 'Month', 'load_lag1', 'load_avg3']

# 获取特征重要性（基于信息增益/分裂次数）
    importances = regr.feature_importance(importance_type='gain')  # 或 'split'
    importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
         }).sort_values(by='Importance', ascending=False)
    



# 准备绘图数据
    verif_plot = data_lgmb[['Electricity demand, daily sum, (GWh)', 'yhat_lgbm']].copy()
    verif_plot.rename(columns={'Electricity demand, daily sum, (GWh)': 'y', 'yhat_lgbm': 'yhat'}, inplace=True)
    verif_plot['train'] = False
    verif_plot.iloc[:train_size, verif_plot.columns.get_loc('train')] = True
    return verif_plot, importance_df

    




In [18]:
# set up the hyper-parameter
params = {}
params['learning_rate'] = 0.005
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mse'
# params['sub_feature'] = 0.5
params['num_leaves'] = 4
params['min_data_in_leaf'] = 30
params['max_depth'] = 5
params['num_threads']=4
params['verbosity']= -1,     # ✅ 控制训练日志：-1 表示不显示
params['verbose']=-1,

In [23]:
# 假设你已经定义好了 lightGBM_train 和 params
# 此代码批量训练五个城市并收集特征重要性
city_list = ['fuzhou', 'singapore', 'la', 'ny', 'sac']
results = {}
importances_list = []

for city in city_list:
    print(f"\n=== Training model for {city.title()} ===")
    verif_plot, importance_df = lightGBM_train(city, params)
    results[city] = verif_plot
    importance_df['City'] = city.title()
    importances_list.append(importance_df)

# 合并所有城市的特征重要性
all_importances = pd.concat(importances_list, ignore_index=True)

# 保存为CSV
all_importances.to_csv('/home/aistudio/results/feature_importance_all_cities.csv', index=False)





=== Training model for Fuzhou ===
-------City: fuzhou--------------
RMSE for Train: 612.8092773207122
RMSE for Test: 637.8255905185582
CVRMSE for Train: 0.0844
CVRMSE for Test: 0.0879

=== Training model for Singapore ===
-------City: singapore--------------
RMSE for Train: 157.39624737881738
RMSE for Test: 167.3026940293324
CVRMSE for Train: 0.0268
CVRMSE for Test: 0.0286

=== Training model for La ===
-------City: la--------------
RMSE for Train: 121.47073564086162
RMSE for Test: 125.3157776247354
CVRMSE for Train: 0.0372
CVRMSE for Test: 0.0395

=== Training model for Ny ===
-------City: ny--------------
RMSE for Train: 663.4548575824889
RMSE for Test: 650.0399805724119
CVRMSE for Train: 0.0367
CVRMSE for Test: 0.0358

=== Training model for Sac ===
-------City: sac--------------
RMSE for Train: 74.17432212082186
RMSE for Test: 73.06868778216172
CVRMSE for Train: 0.0369
CVRMSE for Test: 0.0373
